In [154]:
import Tom_Model as TM
import pandas as pd
import numpy as np
import random

from collections import defaultdict
from copy import deepcopy

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import matplotlib.pyplot as plt
from pandas.plotting import table

from collections import Counter

from tabulate import tabulate

from matplotlib.offsetbox import OffsetImage, AnnotationBbox

In [155]:
def draw_from_pots(round_table, num_pots, group_names):
    
    teams_per_pot = len(group_names)
    
    for val in range(num_pots):
        
        pot = list(round_table.index[0+(val*teams_per_pot):teams_per_pot+(val*teams_per_pot)])
        
        for ind, group in enumerate(group_names):
            
            #pick team from pot
            team = random.choice(pot)
            
            round_table.loc[team, 'Group'] = group
            
            #remove team from pot
            pot = [t for t in pot if t != team]
    
    return round_table

In [156]:
theta = 1.7

In [157]:
iterations = 1

In [158]:
init_table_file = 'OFC_initial.csv'
#current_table_file = 'CONMEBOL_current.csv'

init_sched_file = 'OFC_schedule_init.csv'
#current_sched_file = 'CONMEBOL_schedule_current.csv'

In [159]:
current = input('Do you want to use current Elo, Points, and Schedule? \n Y - Start from Current Spot. \n N - Start from the Beginning. \n')

if current == 'N':
    quals_group_table = pd.read_csv(init_table_file, header=0)
    matches = pd.read_csv(init_sched_file, header=0)
    
# elif current == 'N':
#     quals_group_table = pd.read_csv(init_table_file, header=0)
#     matches = pd.read_csv(init_sched_file, header=0)
    
else:
    print('Please Pick a Valid Response.')

Do you want to use current Elo, Points, and Schedule? 
 Y - Start from Current Spot. 
 N - Start from the Beginning. 
N


In [160]:
quals_group_table = quals_group_table.set_index('Code')

init_group_table = deepcopy(quals_group_table)

In [161]:
quals_group_table

,Round,Group,Points,FIFA,Elo,Flag Code
Code,,,,,,
Cook_Islands,1,SF,0,899.0,680.0,ck
Samoa,1,SF,0,897.0,678.0,ws
Tonga,1,SF,0,749.0,528.0,to
Eastern_Samoa,1,SF,0,693.0,472.0,as
New_Zealand,2,NaN,0,1199.0,1526.0,nz
Solomon_Islands,2,NaN,0,1098.0,1303.0,sb
Papua_New_Guinea,2,NaN,0,1003.0,1136.0,pg
New_Caledonia,2,NaN,0,996.0,1202.0,nc
Tahiti,2,NaN,0,995.0,1151.0,NaN


In [162]:
r1_table = quals_group_table[quals_group_table['Round'] == 1]
r2_table = quals_group_table[quals_group_table['Round'] == 2]

In [163]:
finish_table = pd.DataFrame(0,
                            index = Counter(quals_group_table.index).keys(),
                            columns = ['R1F', 'R2', 'R3', 'Interconfed Playoff', 'Qualify'])

In [164]:
finish_table = finish_table.iloc[0:11, :]

In [165]:
finish_table.loc[r2_table.index[0:7], 'R2'] = iterations

In [166]:
groups = ['A', 'B']

In [167]:
r1_matches = matches[matches['Round'] == 1]
r1_SF = r1_matches[r1_matches['Group'] == 'SF']
r1_F = r1_matches[r1_matches['Group'] == 'F']

r2_matches = matches[matches['Round'] == 2]


In [168]:
r1_SF_matches = len(r1_SF)

In [169]:
for iteration in range(iterations):
    
    #start with R1 semis
    for match in range(2):
            
        #calculate win expectancies
        home_team, home_we, home_elo, away_team, away_we, away_elo, home_FIFA, away_FIFA, home_FIFA_we, away_FIFA_we = TM.calc_match_We_w_FIFA(r1_SF, match, r1_table)
        
        #Determine win probability for each team
        home_wp = TM.davidson_home_wp(home_we, away_we, theta)
        away_wp = TM.davidson_away_wp(home_we, away_we, theta)
        draw_wp = TM.davidson_tie_prob(home_we, away_we, theta)

        r1_table, r1_SF = TM.play_ko_match_w_FIFA(r1_table, r1_SF, match, home_team, home_we, 
                                                       home_elo, home_wp, home_FIFA_we, home_FIFA, away_team, 
                                                       away_we, away_elo, away_wp, away_FIFA_we, away_FIFA, 
                                                       draw_wp)
        
    R1_winners = r1_SF['Winner']
    
    winners_ranks = []
    
    #higher ranked country hosts final
    for winner in R1_winners:
        winners_ranks.append(r1_table.loc[winner, 'FIFA'])
        
    #home and away team based on FIFA ranks
    home = winners_ranks.index(max(winners_ranks))
    away = winners_ranks.index(min(winners_ranks))
    
    r1_F.loc[2, 'Home'] = R1_winners[home]
    r1_F.loc[2, 'Away'] = R1_winners[away]
    
    #R1 Final
    
    #calculate win expectancies
    home_team, home_we, home_elo, away_team, away_we, away_elo, home_FIFA, away_FIFA, home_FIFA_we, away_FIFA_we = TM.calc_match_We_w_FIFA(r1_F, 2, r1_table)

    #Determine win probability for each team
    home_wp = TM.davidson_home_wp(home_we, away_we, theta)
    away_wp = TM.davidson_away_wp(home_we, away_we, theta)
    draw_wp = TM.davidson_tie_prob(home_we, away_we, theta)

    r1_table, r1_F = TM.play_ko_match_w_FIFA(r1_table, r1_F, 2, home_team, home_we, 
                                                   home_elo, home_wp, home_FIFA_we, home_FIFA, away_team, 
                                                   away_we, away_elo, away_wp, away_FIFA_we, away_FIFA, 
                                                   draw_wp)
    
    for team in finish_table.index:
        if team in list(r1_SF['Winner']):
            finish_table.loc[team, 'R1F'] += 1
        else:
            continue
        
        if team in list(r1_F['Winner']):
            finish_table.loc[team, 'R2'] += 1
        else:
            continue  
            
    r1_winner = r1_F.loc[2,'Winner']
    
    #add the R1 winner to the R2 table
    r2_table.rename(index={np.nan:r1_winner},inplace=True)
    r2_table.loc[r1_winner, ['FIFA', 'Elo', 'Flag Code']] = r1_table.loc[r1_winner, ['FIFA', 'Elo', 'Flag Code']]
    
    r2_table = r2_table.sort_values('FIFA', ascending=False)
    
    r2_table = draw_from_pots(r2_table, 4, groups)
            
    quals_group_table = deepcopy(init_group_table)
    
    print(f'Iteration {iteration+1} / {iterations}')

Iteration 1 / 1


/Users/thomasgaudin/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Users/thomasgaudin/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/Users/thomasgaudin/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1797: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

In [170]:
finish_table

,R1F,R2,R3,Interconfed Playoff,Qualify
Cook_Islands,1,1,0,0,0
Samoa,1,0,0,0,0
Tonga,0,0,0,0,0
Eastern_Samoa,0,0,0,0,0
New_Zealand,0,1,0,0,0
Solomon_Islands,0,1,0,0,0
Papua_New_Guinea,0,1,0,0,0
New_Caledonia,0,1,0,0,0
Tahiti,0,1,0,0,0
Vanuatu,0,1,0,0,0


In [171]:
r2_table

,Round,Group,Points,FIFA,Elo,Flag Code
Code,,,,,,
New_Zealand,2,B,0,1199.0,1526.000000,nz
Solomon_Islands,2,A,0,1098.0,1303.000000,sb
Papua_New_Guinea,2,A,0,1003.0,1136.000000,pg
New_Caledonia,2,B,0,996.0,1202.000000,nc
Tahiti,2,A,0,995.0,1151.000000,NaN
Vanuatu,2,B,0,986.0,1108.000000,vu
Fiji,2,A,0,980.0,1197.000000,fj
Cook_Islands,2,B,0,918.5,683.724313,ck


In [172]:
def schedule_matchday(matches, group_table):
    
    
    
    
        
    return matches

In [173]:
def schedule_group_matches(round_matches, groups):
    
    all_group_matches = []
    
    for group in groups:
    
        group_matches = round_matches[round_matches['Group'] == group]
        group_table = round_table[round_table['Group'] == group]
        
        group_size = len(group_table.index)
        group_teams = list(group_table.index)
        
        group_matrix = pd.DataFrame(index=group_teams, names=group_teams)

        all_group_matches.append(group_matches)
        
    round_matches = pd.concat(all_group_matches)
    
    return round_matches
            

In [174]:
r2_matches = schedule_group_matches(r2_matches, groups)

/Users/thomasgaudin/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [175]:
r2_matches

,Match Num,Home,Away,Round,Group,Winner
3,7,Tahiti,Solomon_Islands,2,A,NaN
4,8,Fiji,Papua_New_Guinea,2,A,NaN
7,11,Fiji,Solomon_Islands,2,A,NaN
8,12,Papua_New_Guinea,Tahiti,2,A,NaN
11,15,Papua_New_Guinea,Tahiti,2,A,NaN
12,16,Solomon_Islands,Fiji,2,A,NaN
5,9,New_Caledonia,Cook_Islands,2,B,NaN
6,10,Vanuatu,New_Zealand,2,B,NaN
9,13,New_Zealand,Vanuatu,2,B,NaN
10,14,New_Caledonia,Cook_Islands,2,B,NaN


In [180]:
group_matches = r2_matches[r2_matches['Group'] == 'A']
group_table = r2_table[r2_table['Group'] == 'A']

group_size = len(group_table.index)
group_teams = list(group_table.index)

group_matrix = pd.DataFrame(index=group_teams, columns=group_teams)

for team in group_teams:
    group_matrix.loc[team, team] = 0
    


In [181]:
group_matrix

,Solomon_Islands,Papua_New_Guinea,Tahiti,Fiji
Solomon_Islands,0,NaN,NaN,NaN
Papua_New_Guinea,NaN,0,NaN,NaN
Tahiti,NaN,NaN,0,NaN
Fiji,NaN,NaN,NaN,0
